# etlFFIEC Database Walkabout
There are four tables in the database. They are `dictionary`, `institution`, `period`, and `report`.

Each is detailed below. At the bottom, there's an example of analysis with pandas and SQL.

In [ ]:
import json
from datetime import datetime
from pprint import pprint

import happybase
import pandas
import pandasql
import matplotlib

HBASE = 'node1.hwr.io'  # CHANGE ME
SAMPLE_BANKS = (131034, 720858, 229342, 819172, 65513, 753641)

def stringify_dict(dictionary):
    # Because everything in Hbase is a byte-array,
    # we need to convert the keys/values of retrieved dictionaries into unicode strings
    
    # also, mildly annoying, but the dict keys retain the Hbase column name, pop that off
    return {key.decode('utf-8').split(':', 1).pop(-1): 
            value.decode('utf-8') for key,value in dictionary.items()}

def period_to_datetime(period):
    return datetime.strptime(period, '%m/%d/%Y')

def datetime_to_period(value):
    return datetime.strftime(value, '%m/%d/%Y')

def filter_periods(periods, count, reverse):
    # convert string datestamps to python datetime, sort, return a slice of `count` length
    periods_we_care_about = [period_to_datetime(k) for k in periods]
    periods_we_care_about.sort(reverse=reverse)
    return[datetime_to_period(k) for k in periods_we_care_about[:count]]

# The `dictionary` table

`dictionary` has column `M`, for Metadata. The row-key format for `dictionary` is the MDRM mnemonic and item code appended together, making the MDRM Identifier. It's eight all-uppercase alphanumeric characters, and looks something like this `RCON2170`.

This is the only table not populated from the FFIEC CDR. Instead, it is loaded from a CSV file downloaded from the Federal Reserve. Since this CSV essentially becomes a dependancy of the project, it is dowloaded into the docker image at build time. This table is populated in two scenarios, either when the `--init` flag is passed or when the `--update-metadata` flag is passed. 

In [ ]:
# Scan the entire dictionary table into memory
hbase = happybase.Connection(HBASE)
dictionary = hbase.table('dictionary')
items = [item for item in dictionary.scan()]

mdrm = {}
for row, document in items:
    mdrm[row.decode('utf-8')] = stringify_dict(document)

In [ ]:
for k,v in mdrm.items():
    pprint(k)
    pprint(v)
    break
    
# pay no mind to the encoding!

# The `period` table

`period` has column `I`, for institution. The row-key for `period` is the datestamp format used by the FFIEC to represent a reporting period. This table is used for looking up all institutions which have filed call reports for a given reporting period. Each column value is a json-encoded document of metadata about the institution at the given period.

In [ ]:
hbase = happybase.Connection(HBASE)
period_table = hbase.table('period')

periods = {}
for row, value in period_table.scan():
    document = stringify_dict(value)
    for key in document:
        document[key] = json.loads(document[key])
    
    periods[row.decode('utf-8')] = document


In [ ]:
for k,v in periods.items():
    pprint(k)
    pprint(v)
    break

# The `institution` table
`institution` has column `P`, for period. The row-key format is an [RSSDID](https://www.alacra.com/alacra/outside/lei/info/rssdid.html) which stands for 'Replication Server System Database ID', got all that? This table is used for looking up all reporting periods in which you'll find call reports for the given institution. 


In [ ]:
hbase = happybase.Connection(HBASE)
institution_table = hbase.table('institution')

institutions = {}
for row, value in institution_table.scan():
    document = stringify_dict(value)
    for key in document:
        document[key] = json.loads(document[key])
    
    institutions[row.decode('utf-8')] = document

In [ ]:
for  k,v in institutions.items():
    pprint(k)
    pprint(v)
    break

In [ ]:
hbase = happybase.Connection('node1.hwr.io', timeout=300000)  # connection with 5-minute timeout
reports_table = hbase.table('report')

# One usability improvement would be to reverse the order of the report period datastamp,
# which would allow for scanning institution by year, like `row_prefix=b'534242-2017'`

# having period formatted '%Y-%m-%d' would suit sqlite's date function as well

# It may also be better to create a document for each reported metric, 
# in the same fashion as the `institution` and `period` tables,
# rather than N column entries for N fields

# Regardless, this is the time to go get coffee
call_reports = {}
for bank in SAMPLE_BANKS:
    prefix = bytes('{}-'.format(bank), 'utf-8')
    for row, report in reports_table.scan(row_prefix=prefix):
        call_reports[row.decode('utf-8')] = stringify_dict(report)

In [ ]:
for k,v in call_reports.items():
    pprint(k)
    pprint(v)
    break

In [ ]:
# dig for interesting asset-related reports, enrich the item_name

possible_relevent_fields = {}

for key, value in mdrm.items():
    item_type = value['item_type']
    item_name = value['item_name'].lower()
    if 'total asset' in item_name:   
        possible_relevent_fields[key] = '{} {}'.format(item_type, item_name)

In [ ]:
pprint(possible_relevent_fields[)

In [ ]:
# process call_reports into a time-series
data = []
for row_key, call_report in call_reports.items():  
    rssd, period = row_key.split('-')
    
    for key, value in call_report.items():
        mdrm_id, name = key.split(':')
        
        if mdrm_id not in possible_relevent_fields:
            continue
        
        if name != 'value':
            continue
                
        try:
            value = float(value)
            if value.is_integer():
                value = int(value)
        except ValueError:
            pass

        if isinstance(value, str) and not value:
            value = None
            
        data.append({'institution': institutions[rssd][period]['Name'],
                     'mdrm': mdrm_id,
                     'type': mdrm[mdrm_id]['item_type'],
                     'name': mdrm[mdrm_id]['item_name'],
                     'rssd': int(rssd), 
                     'value': value,
                     'period': period_to_datetime(period)})

In [ ]:
for k,v in data.items():
    pprint(k)
    pprint(v)
    break


# Ok, so can you answer the question?
> how has asset growth over time compared with other / similar banks?

We'll look at MDRM ID RCON2170, derrived total assets.

In [16]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
reports_frame = pandas.DataFrame(data=data)

In [17]:
# let's compare 131034, 819172, 720858, 229342, 819172, 753641
# it seems there are no RCON2170 values for 65513
pysqldf('''select rssd, institution, type, name, strftime('%Y', period) as year, sum(value) as "sum"
                  from reports_frame where mdrm="RCON2170"
                  GROUP BY year, rssd
                  ORDER BY rssd, year''')

,rssd,institution,type,name,year,sum
0,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2001,437415.0
1,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2002,493609.0
2,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2003,536692.0
3,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2004,584885.0
4,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2005,663896.0
5,131034,"CITIZENS BANK OF OVIEDO, THE",derived,TOTAL ASSETS,2006,731743.0
6,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2007,751258.0
7,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2008,961270.0
8,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2009,915913.0
9,131034,CITIZENS BANK OF FLORIDA,derived,TOTAL ASSETS,2010,914478.0


In [18]:
pysqldf('''select institution, type, name, mdrm, rssd, strftime('%Y', period) as year, sum(value)
                  from reports_frame where mdrm="RCFD5320" 
                  GROUP BY year, rssd
                  ORDER BY year, rssd''')

,institution,type,name,mdrm,rssd,year,sum(value)
0,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2001,488178.0
1,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2002,458185.0
2,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2003,383452.0
3,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2004,255093.0
4,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2005,182524.0
5,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2006,113664.0
6,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2007,85282.0
7,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2008,318283.0
8,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2009,927706.0
9,DELTA NATIONAL BANK AND TRUST COMPANY,reported,TOTAL ASSETS (0% RISK-WEIGHT),RCFD5320,65513,2010,935286.0


In [ ]:
pysqldf('''select institution, type, name, mdrm, rssd, strftime('%Y', period) as year, sum(value)
                  from reports_frame
                  GROUP BY year, rssd
                  ORDER BY year, rssd, mdrm''')